## Import necessary Python packages

In [69]:
import os
import pickle
import multirecording_spikeanalysis as spike
import numpy as np
# import pandas as pd # use pandas for more functionality
import modin.pandas as pd # use modin to speed things up
import matplotlib.pyplot as plt
import seaborn as sns

## Define working directory relative to repository

In [70]:
# get working directory as paerent directory of current directory
cwd = os.getcwd()
pwd = os.path.dirname(cwd)

## Import .pkl files

In [71]:
# import pickle files given by lab
with open(pwd + '/01_Raw_data/phase2_collection.pkl', 'rb') as f:
    phase2 = pickle.load(f)

# Not usoing phase 3 data
# with open(pwd + '/01_Raw_data/phase3_collection.pkl', 'rb') as f:
#     phase3 = pickle.load(f)

## Notes and reminders about data

In [84]:
# show the subject to which each recording belongs to
# .subject
# shows the type of unit/neuron (we only use good units in unit_timestamps)
# .labels_dict
# all the timestamps for all units
# .timestamps_var
# start here (Only use these for now)
# all the timestamps for each unit
# .unit_timestamps 
# the behaviour labels for each timestamp with a starting and ending time
# .event_dict

In [75]:
# get an aray of timestamps for each behaviour
# each unit is seen as an indipendent case? or all units of the same indiviudal are seen as one case? -> ask Meghan >> see if each individual has same number of units
# calcualte the distance bewteen timeseries of each case and compare them bewteen behaviours without dimensionality reduction (treat it as a vector) >>> look up ways to comapre time series data and calcualte distance bewteen time series data
# draw conclusions of simialr behaviours are. 

# Qs meghan: is each recording the same length around the event? Not the case for random stuff I assume.
# Dtw, mahalonobis, mutual information to allign all cases
# make graph with naode as case and edge as similarity
# cluster and compare clusters to behaviour/indvidual/unit


# Extract Metadata from .pkl files into a dataframe

In [83]:
# create metadata dataframe
metadata_df = pd.DataFrame() # create empty dataframe
temp_df_lst = [] # create empty list to store dataframes
for i, j in phase2.collection.items(): # loop through each recording
    temp_df = pd.DataFrame() # create empty dataframe
    start_time_lst = [] # create empty list to store start times
    end_time_lst = [] # create empty list to store end times
    behavior_lab_lst = [] # create empty list to store behavior labels
    for k, v in j.event_dict.items(): # loop through each behavior
        start_time_lst += list(v[:,0]) # add start times to list
        end_time_lst += list(v[:,1]) # add end times to list
        behavior_lab_lst += list([k] * len(v)) # add behavior labels to list
    temp_df['behavior_label'] = behavior_lab_lst # add behavior labels to dataframe
    temp_df['start_time'] = start_time_lst # add start times to dataframe
    temp_df['end_time'] = end_time_lst # add end times to dataframe
    temp_df['collection'] = i # add recording name to dataframe
    temp_df['subject'] = j.subject # add subject to dataframe
    temp_df_lst.append(temp_df) # add dataframe to list
metadata_df = pd.concat(temp_df_lst) # concatenate all dataframes in list to one dataframe

# show dataframe
metadata_df    

,behavior_label,start_time,end_time,collection,subject
0,acquisition,15167.0,24733.0,20230803_101331_1_merged.rec,1.1
1,acquisition,27433.0,33833.0,20230803_101331_1_merged.rec,1.1
2,acquisition,69667.0,70367.0,20230803_101331_1_merged.rec,1.1
3,acquisition,84100.0,85400.0,20230803_101331_1_merged.rec,1.1
4,acquisition,118067.0,122367.0,20230803_101331_1_merged.rec,1.1
...,...,...,...,...,...
62,exposure 3,1833033.0,1833667.0,20230818_133620_1_merged.rec,1.4
63,exposure 3,1860933.0,1861533.0,20230818_133620_1_merged.rec,1.4
64,exposure 3,1866333.0,1869033.0,20230818_133620_1_merged.rec,1.4
65,exposure 3,1971667.0,1972167.0,20230818_133620_1_merged.rec,1.4


In [ ]:
phase2.collection['20230803_101331_1_merged.rec'].unit_timestamps[2]